In [ ]:
# 선호도
def itemCF(request, user_uuid):#사용자1, 아이템1
    
    result = item_collaborativeFiltering(user_uuid) #str:dict
    # json 형식으로 두 가지 종류 추천을 반환한다
    # 아이템:안먹은 유사아이템
    # 사용자 > 안먹은 아이템  각각 하나씩

    return HttpResponse(result)


In [ ]:
# emotion_keyword_to_vec를 만든다. 인코딩용. {‘감정’: 인코딩결과} 형태

# 8가지 감정 키워드
emotion_keyword = ["anger","contempt","disgust","fear","happiness","neutral","sadness","surprise"]

emotion_df = pd.DataFrame({ 'emotion_keyword':emotion_keyword })
emotion_df['emotion_encoding'] = pd.factorize(emotion_df['emotion_keyword'])[0] 

#print(emotion_df.values)
emotion_keyword_to_vec = {} #딕셔너리로 저장
for keyword,encoding in emotion_df.values:
    emotion_keyword_to_vec[keyword] = encoding



In [ ]:
# order에 있는 데이터 가져온다
# 최종적으로 Pandas DataFrame 으로 변환한다

# Firestore에 접근
docs = firestore.client().collection(u'order').stream()

doc_list = []
for doc in docs:
    doc_list_row = []
    
    # 메뉴 이름 - 레이블
    # 데이터가 거의 없는 문제로 [메뉴이름+상세옵션] 에서 [메뉴이름]으로 학습 레이블 변경
    doc_list_row.append(doc.get('items')[0].get('name'))

    # 날씨
    for key in doc.get('weather').keys():
        doc_list_row.append(doc.get('weather').get(key))
    
    # 감정
    emotion_key_list = ''
    for key in doc.get('emotion').keys():
        emotion_key_list += str(emotion_keyword_to_vec[key])  # 감정 데이터 전처리
    doc_list_row.append(emotion_key_list)
    
    # 날짜
    list_date = doc.get('today').split(' ')
    doc_list_row.append(list_date[0])
    doc_list_row.append(list_date[1])
    
    doc_list.append(doc_list_row)

doc_df = pd.DataFrame(doc_list, columns=['item_name','w_humidity','w_temp','w_speed','emotion','date','time'])
doc_df['emotion'] = doc_df['emotion'].astype(int) # 감정 데이터 전처리 str to int


In [ ]:
# 메뉴 이름
doc_df['item_name'], item_name_restore = pd.factorize(doc_df['item_name']) # encoding 메뉴이름

# date > 요일 > 주중/주말
doc_df['num_of_days'] = doc_df['date'].apply(lambda x: date_to_VIP(x))
doc_df['work_or_free'] = doc_df['num_of_days'].apply(lambda x: num_of_days_to_work_or_free(x))

# time > 아침(~11)/오후(~16)/저녁(16~)
doc_df['time_to_3'] = doc_df['time'].apply(lambda x: time_to_3(x))


In [ ]:
# DataFrame 전처리
# 날 수 세는 함수

def date_to_VIP(date):
    year,month,day = date.split('-')
    year = int(year)
    month = int(month)
    day = int(day)
    preYear = year-1
    num_of_days = preYear*365 + (preYear/4 - preYear/100 + preYear/400)
    
    monthArray = [31,28,31,30,31,30,31,31,30,31,30,31]
    for m in range(month):
        num_of_days += monthArray[m]
    
    cond = (month>=3 and (year%4==0 and year%100!=0 or year%400==0))
    if cond == True:
        num_of_days += 1
    
    num_of_days += day
    
    return int(num_of_days)